# Initial prediction
So for now we've come to conclusion that PassengerID and Name features can be safely dropped. Now let's get the initial model and prediction now to set a benchmark for our work. We will discard all non-numeric features for now, except for Sex and Embarkation.

## Retrieving data

In [1]:
import pandas as pd
# 1. Read data and form X and y
train_df = pd.read_csv('train.csv')
X = train_df.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'], 1)
y = train_df['Survived']
# 2. Conver male, female and C, Q, S into categorials
mapping = {'male': 0, 'female': 1, 'C': 0, 'Q': 1, 'S': 2}
X.replace({'Sex': mapping, 'Embarked': mapping}, inplace=True)
# 3. Replace NaNs in Age
X.fillna(X['Age'].mean(), inplace=True)
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,22.0,1,0,7.2500,2.0
1,1,1,38.0,1,0,71.2833,0.0
2,3,1,26.0,0,0,7.9250,2.0
3,1,1,35.0,1,0,53.1000,2.0
4,3,0,35.0,0,0,8.0500,2.0


## Choosing metrics
Titanic competition on Kaggle requires usage of **accuracy** metrics, so this will be the main metrics we use.



## Making the initial, fast estimation

First of all we will use **KNN** to get some off the bat understanding:

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
knn = KNeighborsClassifier(n_neighbors=8, weights='distance')
scores = cross_val_score(knn, X, y, scoring='accuracy', cv=15)
print("Mean cross-validated score is:", scores.mean())

Mean cross-validated score is: 0.748751217612


So right off the bat we got **~0.75** accuracy. This is an expected accuracy for KNN though, because data is not normalized and KNN with 'distance' setting is kind of sensitive to that: if Sex is 0 or 1, Fare can be 7, 8, 71, 53, so in this case KNN would be skewed towards Fare.
Let's try the same with LogisticRegression:

In [3]:
from sklearn.linear_model import LogisticRegression
# Logistic regression
log_regression = LogisticRegression()
scores = cross_val_score(log_regression, X, y, scoring='accuracy', cv=10)
print("Cross validated scores are:", scores)
print("Mean is:", scores.mean())

Cross validated scores are: [ 0.77777778  0.78888889  0.7752809   0.80898876  0.79775281  0.76404494
  0.79775281  0.7752809   0.84269663  0.81818182]
Mean is: 0.794664623766


In [4]:
from sklearn.linear_model import LinearRegression
# Linear regression
lin_regression = LinearRegression()

# AUC score
scores = cross_val_score(lin_regression, X, y, scoring='roc_auc', cv=10)
print("--------- ROC AUC score ---------")
# print("Cross validated scores are:", scores)
print("Mean CV score is:", scores.mean())

# Accuracy score via prediction rounding
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)
lin_regression.fit(X_train, y_train)
y_pred = lin_regression.predict(X_test)
print("--------- Accuracy score ---------")
print("Accuracy for some random split:", accuracy_score(y_test, y_pred.round()))

--------- ROC AUC score ---------
Mean CV score is: 0.849027774844
--------- Accuracy score ---------
Accuracy for some random split: 0.856502242152


Okay so at this point it seems that KNN is not particulary good here, and LogisticRegression might be the model of choice **for now**.

**Here another models will be added and tested as I learn them.**

# Up-to-date Kaggle score
As of 23.11.17 the score 

In [5]:
from sklearn.ensemble import AdaBoostClassifier
ada=AdaBoostClassifier(n_estimators=200,random_state=0,learning_rate=0.1)
result=cross_val_score(ada,X,y,cv=10,scoring='accuracy')
print('The cross validated score for AdaBoost is:',result.mean())

The cross validated score for AdaBoost is: 0.807049426853


In [9]:
"""
================================== OUTPUT HERE IS: ==================================
Fitting 3 folds for each of 120 candidates, totalling 360 fits

[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:  4.8min finished

0.805836139169
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.7, n_estimators=600, random_state=None)
"""



# from sklearn.model_selection import GridSearchCV

# n_estimators=list(range(100,1100,100))
# learn_rate=[0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
# hyper={'n_estimators':n_estimators,'learning_rate':learn_rate}
# gd=GridSearchCV(estimator=AdaBoostClassifier(),param_grid=hyper,verbose=True)
# gd.fit(X,y)
# print(gd.best_score_)
# print(gd.best_estimator_)

"\n================================== OUTPUT HERE IS: ==================================\nFitting 3 folds for each of 120 candidates, totalling 360 fits\n\n[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:  4.8min finished\n\n0.805836139169\nAdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,\n          learning_rate=0.7, n_estimators=600, random_state=None)\n"

In [8]:
ada = AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.7, n_estimators=600, random_state=None)
result=cross_val_score(ada,X,y,cv=10,scoring='accuracy')
print('The cross validated score for AdaBoost is:',result.mean())

The cross validated score for AdaBoost is: 0.812680172512
